## QHyper usecase

### Create instance of problem

In [1]:
from QHyper.problems import KnapsackProblem, TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one penalty weights, and objective function also
# requires one

if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and
    # value 2 - (1, 2), and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, item_values=[1, 1, 3], item_weights=[1, 1, 2])

    problem_config = {  # This is going to be used later
        'type': 'knapsack',
        'max_weight': 2,
        'item_values': [1, 1, 3],
        'item_weights': [1, 1, 2],
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]
    penalty = 2

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(number_of_cities=3)

    problem_config = {  # This is going to be used later
        'type': 'tsp',
        'number_of_cities': 3,
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]
    penalty = -1


In [2]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")


Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -1.0, ('x1',): -1.0, ('x2',): -3.0}))
Constraints:
    Polynomial(terms=defaultdict(<class 'float'>, {('x3',): -1.0, ('x4',): -1.0, (): 1.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))
    Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -1.0, ('x1',): -1.0, ('x2',): -2.0, ('x3',): 1.0, ('x4',): 2.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))


### Use VQA to solve knapsack problem

In [1]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)
from QHyper.solvers import solver_from_config
from numpy import pi

# While we could use problem instace to create solver, the better approach
# is to use config, because it is easier to save it in JSON file and load it later

# solver_config = {
#     "solver": {
#         "type": "vqa",
#         "optimizer": {
#             "type": "scipy",
#             "maxfun": 200,
#             "bounds": [(0, 2 * pi)] * 10
#         },
#         "pqc": {
#             "type": "wfqaoa",
#             "layers": 5,
#             "limit_results": 20,
#             "penalty": penalty,
#             "backend": "default.qubit",
#         },
#         "params_inits": params_config
#     },
#     "problem": problem_config
# }

# solver_config = {
#     "solver": {
#         "type": "dqm",
#         "time": 5
#     },
#     "problem": problem_config
# }
problem_config = {  # This is going to be used later
       'type': 'knapsack',
        'max_weight': 2,
        'item_values': [1, 1, 3],
        'item_weights': [1, 1, 2],
    }
solver_config = {
    'problem': problem_config,
    # 'solver': {
    #     'name': 'DQM',
    #     'category': 'quantum_annealing',
    #     'platform': 'dwave',
    #     'time': 5,
    #     # 'num_reads': 100,
    #     "token": "DEV-c89566dc7b9419a5ddbc714bf36557ee3dcf3a4a"
    # },
    'solver': {
        'name': 'Gurobi',
        'category': 'classical',
        'platform': 'gurobi',
    }
    # 'solver': {
    #     'name': 'qaoa',
    #     'category': 'gate_based',
    #     'platform': 'pennylane',
    #     'gamma': {
    #         'min': [0] * 5,
    #         'max': [2*pi] * 5,
    #         'init': [0.5] * 5,
    #     },
    #     'beta': {
    #         'min': [0] * 5,
    #         'max': [2*pi] * 5,
    #         'init': [1] * 5,
    #     },
    #     'layers': 5,
    #     'penalty_weights': [1, 2, 2],
    #     # 'penalty_weights': {
    #     #     'min': [1, 1, 1],
    #     #     'max': [3, 3, 3],
    #     #     'init': [1, 2, 2],
    #     # },
    #     # 'penalty': 3,
    #     'optimizer': {
    #         "type": "qml",
    #         "name": "adam"
    #         # "maxfun": 200,
    #     },
    # },
    # "hyper_optimizer": {
    #     'optimizer': {
    #         "type": "random",
    #         "processes": 5,
    #         "number_of_samples": 5,
    #     },
    #     'weights': {
    #         'min': [1, 1, 1],
    #         'max': [3, 3, 3],
    #         # 'init': [2, 2, 2],
    #         # 'step': [1, 1, 1]
    #     },
    #     # 'gamma': {
    #     #     'min': [0] * 5,
    #     #     'max': [2*pi] * 5,
    #     #     'init': [0.5] * 5,
    #     # },
    #     # 'beta': {
    #     #     'min': [0] * 5,
    #     #     'max': [2*pi] * 5,
    #     #     'init': [1] * 5,
    #     # },
    # }
}
vqa = solver_from_config(solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# NOT RECOMMENDED

# from QHyper.solvers.vqa import VQA
# from QHyper.solvers.vqa.pqc import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, HQAOA(layers=5, penalty=penalty, backend='default.qubit'),
#           ScipyOptimizer(maxfun=200))
solver_results_1 = vqa.solve()

Imported <class 'test.solver1'> from /home/tmek/QHyper/custom/test.py as solver1
Imported <class 'test.test'> from /home/tmek/QHyper/custom/test.py as test


In [2]:
solver_results_1

SolverResult(probabilities=rec.array([(0, 0, 0, 1, 1, 1.)],
          dtype=[('x0', '<i4'), ('x1', '<i4'), ('x3', '<i4'), ('x2', '<i4'), ('x4', '<i4'), ('probability', '<f8')]), params={}, history=[])

In [3]:
best_res = solver_results_1

In [4]:
best_res = vqa.run_solver(solver_results_1.params)

AttributeError: 'Gurobi' object has no attribute 'run_solver'

In [5]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation

# solver_results = solver_results_1

print("Evaluation:")
print(weighted_avg_evaluation(
    best_res.probabilities, vqa.problem.get_score,
    penalty=0, limit_results=30, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    best_res.probabilities, limit_results=100)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, vqa.problem.get_score, penalty=0)

for rec in results_with_evaluation:
    print(f"Result: {rec}, "
          f"Prob: {rec['probability']:.5}, "
          f"Evaluation: {rec['evaluation']:.5}")

Evaluation:
-3.0
Sort results:
Result: (0, 0, 1, 0, 1, 1., -3.), Prob: 1.0, Evaluation: -3.0


#### Using hyper optimizers

In [ ]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change penalty_weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "optimizer": {
            "type": "scipy",
            "maxfun": 200,
            "bounds": [(0, 2 * pi)] * 10
        },
        "pqc": {
            "type": "wfqaoa",
            "layers": 5,
            "limit_results": 20,
            "penalty": penalty,
        },
        "params_inits": params_config,
        "hyper_optimizer": {
            "type": "random",
            "processes": 5,
            "number_of_samples": 100,
            "bounds": hyper_optimizer_bounds,
        }
    },
    "problem": problem_config
}
vqa = solver_from_config(solver_config)


In [ ]:
solver_results = vqa.solve()

print("Solver results:")
print(f"Probabilities: {solver_results.probabilities}")
print(f"Best params: {solver_results.params}")


Solver results:
Probabilities: {'00000': 0.036215643738056184, '00001': 0.009732461820058054, '00010': 0.01512752159177665, '00011': 0.004898970085671026, '00100': 0.009811070774507602, '00101': 0.009995230222961251, '00110': 0.17020709388664948, '00111': 0.009835147114974447, '01000': 0.009633808259437516, '01001': 6.551348341363583e-05, '01010': 0.06818393104282872, '01011': 0.0032310466245177418, '01100': 0.008494761842195329, '01101': 0.13505509463529042, '01110': 0.01678625146275098, '01111': 0.01106622875202749, '10000': 0.00963380825943751, '10001': 6.551348341363564e-05, '10010': 0.06818393104282863, '10011': 0.0032310466245177435, '10100': 0.00849476184219533, '10101': 0.13505509463529033, '10110': 0.01678625146275097, '10111': 0.011066228752027488, '11000': 0.018086704657015597, '11001': 0.14481572304362988, '11010': 0.005243829793381593, '11011': 0.013867749560177636, '11100': 0.009298811833917654, '11101': 0.0028672422311025393, '11110': 0.010112918453326875, '11111': 0.024

In [ ]:
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.probabilities, limit_results=10)

results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


Evaluation:
-2.241798915063301
Sort results:
Result: 00110, Prob: 0.17021, Evaluation: -1
Result: 11001, Prob: 0.14482, Evaluation: -4
Result: 01101, Prob: 0.13506, Evaluation: -3
Result: 10101, Prob: 0.13506, Evaluation: -3
Result: 01010, Prob: 0.068184, Evaluation: -2
Result: 10010, Prob: 0.068184, Evaluation: -2
Result: 00000, Prob: 0.036216, Evaluation: 0
Result: 11111, Prob: 0.024851, Evaluation: 2
Result: 11000, Prob: 0.018087, Evaluation: 2
Result: 01110, Prob: 0.016786, Evaluation: 2


In [ ]:
solver_config = {
    'problem': {
        'type': 'tsp',
        'number_of_cities': 3,
    },
    'solver': {
        'name': 'QAOA',
        'category': 'gate_based',
        'platform': 'pennylane',
        'angles': {
            'min': [[0]*5, [0]*5],
            'max': [[2*pi]*5, [pi]*5],
            'init': [[0.5]*5, [1]*5],
        },
        'layers': 5,
        'penalty_weights': [1, 2, 2],
        'optimizer': {
            "type": "scipy",
            "maxfun": 200,
            "bounds": [(0, 2 * pi)] * 10
        },
    },
    "hyper_optimizer": {
        'optimizer': {
            "type": "random",
            "processes": 1,
            "number_of_samples": 2,
        },
        'penalty_weights': {
            'min': [1, 1, 1],
            'max': [10, 10, 10],
            'init': [2, 2, 2]
        },
        'angles': {
            'min': [2] * 10,
            'max': [4] * 10,
            'init': [3] * 10,
            # 'steps': []
        }
    }
}